In [1]:
import requests
import base64
import os
import jwt
import time
import sqlite3
import json
import re
import concurrent.futures

def get_jwt():
    """fetch the jwt token object"""
    headers = {
        'User-Agent': 'Jobsuche/2.9.2 (de.arbeitsagentur.jobboerse; build:1077; iOS 15.1.0) Alamofire/5.4.4',
        'Host': 'rest.arbeitsagentur.de',
        'Connection': 'keep-alive',
        'Content-Type': 'application/x-www-form-urlencoded; charset=utf-8',
    }

    data = {
      'client_id': 'c003a37f-024f-462a-b36d-b001be4cd24a',
      'client_secret': '32a39620-32b3-4307-9aa1-511e3d7f48a8',
      'grant_type': 'client_credentials'
    }

    response = requests.post('https://rest.arbeitsagentur.de/oauth/gettoken_cc', headers=headers, data=data, verify=True) #, verify=True

    return response.json()

def search2(jwt, page):
    params = (
        ('angebotsart', '2'),
        ('page', str(page)),
        #('wo', where),
    )

    headers = {
       'User-Agent': 'Jobsuche/2.9.2 (de.arbeitsagentur.jobboerse; build:1077; iOS 15.1.0) Alamofire/5.4.4',
        'Host': 'rest.arbeitsagentur.de',
        'OAuthAccessToken': jwt,
        'Connection': 'keep-alive',
    }

    response = requests.get('https://rest.arbeitsagentur.de/jobboerse/jobsuche-service/pc/v4/app/jobs',
                            headers=headers, params=params, verify=True)   #, verify=True
    return response.json()

def search4(jwt, page):
    params = (
        ('angebotsart', '4'),
        ('page', str(page)),
        #('wo', where),
    )

    headers = {
       'User-Agent': 'Jobsuche/2.9.2 (de.arbeitsagentur.jobboerse; build:1077; iOS 15.1.0) Alamofire/5.4.4',
        'Host': 'rest.arbeitsagentur.de',
        'OAuthAccessToken': jwt,
        'Connection': 'keep-alive',
    }

    response = requests.get('https://rest.arbeitsagentur.de/jobboerse/jobsuche-service/pc/v4/app/jobs',
                            headers=headers, params=params, verify=True)   #, verify=True
    return response.json()

def search34(jwt, page):
    params = (
        ('angebotsart', '34'),
        ('page', str(page)),
        #('wo', where),
    )

    headers = {
       'User-Agent': 'Jobsuche/2.9.2 (de.arbeitsagentur.jobboerse; build:1077; iOS 15.1.0) Alamofire/5.4.4',
        'Host': 'rest.arbeitsagentur.de',
        'OAuthAccessToken': jwt,
        'Connection': 'keep-alive',
    }

    response = requests.get('https://rest.arbeitsagentur.de/jobboerse/jobsuche-service/pc/v4/app/jobs',
                            headers=headers, params=params, verify=True)   #, verify=True
    return response.json()

def search1(jwt, page):
    params = (
        ('angebotsart', '1'),
        ('page', str(page)),
        #('wo', where),
    )

    headers = {
       'User-Agent': 'Jobsuche/2.9.2 (de.arbeitsagentur.jobboerse; build:1077; iOS 15.1.0) Alamofire/5.4.4',
        'Host': 'rest.arbeitsagentur.de',
        'OAuthAccessToken': jwt,
        'Connection': 'keep-alive',
    }

    response = requests.get('https://rest.arbeitsagentur.de/jobboerse/jobsuche-service/pc/v4/app/jobs',
                            headers=headers, params=params, verify=True)   #, verify=True
    return response.json()


def job_details(jwt, job_ref):

    headers = {
        'User-Agent': 'Jobsuche/2.9.3 (de.arbeitsagentur.jobboerse; build:1078; iOS 15.1.0) Alamofire/5.4.4',
        'Host': 'rest.arbeitsagentur.de',
        'OAuthAccessToken': jwt,
        'Connection': 'keep-alive',
    }

    response = requests.get(
        f'https://rest.arbeitsagentur.de/jobboerse/jobsuche-service/pc/v2/jobdetails/{(base64.b64encode(job_ref.encode())).decode("UTF-8")}',
        headers=headers, verify=True)  #, verify=False

    return response.json()

def create_table():
    conn = sqlite3.connect('weiterbildung_analysis.db')
    cursor = conn.cursor()

    # Create the job_listing table if it doesn't exist
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS Jobsuche_Details (
            aktuelleVeroeffentlichungsdatum TEXT,
            laufzeitBis TEXT,
			angebotsart INTEGER,
            arbeitgeber TEXT,
            branchengruppe TEXT,
			branche TEXT,
            arbeitsorte_land TEXT,
			arbeitsorte_region TEXT,
			arbeitsorte_plz INTEGER,
			arbeitsorte_ort TEXT,
            arbeitsorte_koordinaten_lat REAL,
            arbeitsorte_koordinaten_lon REAL,
            arbeitszeitmodelle TEXT,
            befristung TEXT,
			uebernahme TEXT,
			betriebsgroesse TEXT,
			eintrittsdatum TEXT,
			ersteVeroeffentlichungsdatum TEXT,
			allianzpartner  TEXT,
			allianzpartnerUrl TEXT,
			titel TEXT,
			beruf TEXT,
			modifikationsTimestamp TEXT,
			stellenbeschreibung TEXT,
			refnr TEXT PRIMARY KEY,
			fuerFluechtlingeGeeignet BOOLEAN,
			nurFuerSchwerbehinderte BOOLEAN,
			anzahlOffeneStellen TEXT,
            arbeitgeber_land TEXT,
            arbeitgeber_region TEXT,
            arbeitgeber_plz INTEGER,
            arbeitgeber_ort TEXT,
            arbeitgeber_strasse TEXT,
            arbeitgeber_strasseHausnummer TEXT,
            fertigkeiten_hierarchieName TEXT,
            fertigkeiten_auspraegungen TEXT,
            mobilitaet_reisebereitschaft TEXT,
            fuehrungskompetenzen_hatVollmacht BOOLEAN,
            fuehrungskompetenzen_hatBudgetverantwortung BOOLEAN,
            verguetung TEXT,
            arbeitgeberdarstellungUrl TEXT,
            arbeitgeberdarstellung TEXT,
            hauptDkz TEXT,
            istBetreut BOOLEAN,
            istPrivateArbeitsvermittlung BOOLEAN,
            istZeitarbeit BOOLEAN,
            istGoogleJobsRelevant BOOLEAN,
            anzeigeAnonym BOOLEAN
        )
    ''')

def insert_data_into_db(data):
    conn = sqlite3.connect('weiterbildung_analysis.db')
    cursor = conn.cursor()

    for entry in data:
        aktuelleVeroeffentlichungsdatum = entry.get("aktuelleVeroeffentlichungsdatum", "")
        laufzeitBis = entry.get("laufzeitBis", "")
        angebotsart = entry.get("angebotsart", "")
        arbeitgeber = entry.get("arbeitgeber", "")
        branchengruppe = entry.get("branchengruppe", "")
        branche = entry.get("branche", "")
        
        arbeitsorte = entry.get("arbeitsorte", [])
        for arbeitsort in arbeitsorte:
            arbeitsorte_land = arbeitsort.get("land")
            arbeitsorte_region = arbeitsort.get("region")
            arbeitsorte_plz = arbeitsort.get("plz")
            arbeitsorte_ort = arbeitsort.get("ort")
            arbeitsorte_koordinaten_lat = arbeitsort.get("koordinaten", {}).get("lat")
            arbeitsorte_koordinaten_lon = arbeitsort.get("koordinaten", {}).get("lon")

        arbeitszeitmodelle = entry.get("arbeitszeitmodelle", [])
        arbeitszeitmodelle_str = re.sub(r'[\[\]"]', '', str(json.dumps(arbeitszeitmodelle)))

        befristung = entry.get("befristung", "")
        uebernahme = entry.get("uebernahme", "")
        betriebsgroesse = entry.get("betriebsgroesse", "")
        eintrittsdatum = entry.get("eintrittsdatum", "")
        ersteVeroeffentlichungsdatum = entry.get("ersteVeroeffentlichungsdatum", "")
        allianzpartner = entry.get("allianzpartner", "")
        allianzpartnerUrl = entry.get("allianzpartnerUrl", "")
        titel = entry.get("titel", "")
        beruf = entry.get("beruf", "")
        modifikationsTimestamp = entry.get("modifikationsTimestamp", "")

        stellenbeschreibung = entry.get("stellenbeschreibung", "")        
        truncated_stellenbeschreibung = re.sub(r'[\[\]\-"*]', '', str(json.dumps(stellenbeschreibung[:100])))

        refnr = entry.get("refnr", "")
        fuerFluechtlingeGeeignet = entry.get("fuerFluechtlingeGeeignet", "")
        nurFuerSchwerbehinderte = entry.get("nurFuerSchwerbehinderte", "")
        anzahlOffeneStellen = entry.get("anzahlOffeneStellen", "")

        arbeitgeber_adresse = entry.get("arbeitgeberAdresse", {})
        arbeitgeber_land = arbeitgeber_adresse.get("land")
        arbeitgeber_region = arbeitgeber_adresse.get("region")
        arbeitgeber_plz = arbeitgeber_adresse.get("plz")
        arbeitgeber_ort = arbeitgeber_adresse.get("ort")
        arbeitgeber_strasse = arbeitgeber_adresse.get("strasse")
        arbeitgeber_strasseHausnummer = arbeitgeber_adresse.get("strasseHausnummer")

        fertigkeiten_hierarchieName = ""
        fertigkeiten_auspraegungen = ""

        fertigkeiten = entry.get("fertigkeiten", [])
        for fertigkeit in fertigkeiten:
            fertigkeiten_hierarchieName = fertigkeit.get("hierarchieName")
            fertigkeiten_auspraegungen = json.dumps(fertigkeit.get("auspraegungen", {}))

        # Extract "mobilitaet" data
        mobilitaet = entry.get("mobilitaet", {})
        mobilitaet_reisebereitschaft = mobilitaet.get("reisebereitschaft")

        # Extract "fuehrungskompetenzen" data
        fuehrungskompetenzen = entry.get("fuehrungskompetenzen", {})
        fuehrungskompetenzen_hatVollmacht = fuehrungskompetenzen.get("hatVollmacht")
        fuehrungskompetenzen_hatBudgetverantwortung = fuehrungskompetenzen.get("hatBudgetverantwortung")

        verguetung = entry.get("verguetung", "")
        arbeitgeberdarstellungUrl = entry.get("arbeitgeberdarstellungUrl", "")

        arbeitgeberdarstellung = entry.get("arbeitgeberdarstellung", "")
        truncated_arbeitgeberdarstellung = arbeitgeberdarstellung[:100]  # Truncate to 100 characters

        hauptDkz = entry.get("hauptDkz", "")
        istBetreut = entry.get("istBetreut", "")
        istGoogleJobsRelevant = entry.get("istGoogleJobsRelevant", "")
        anzeigeAnonym = entry.get("anzeigeAnonym", "")
        istPrivateArbeitsvermittlung = entry.get("istPrivateArbeitsvermittlung", "")
        istZeitarbeit = entry.get("istZeitarbeit", "")

        # Check if the refnr already exists in the table
        cursor.execute('''SELECT COUNT(*) FROM Jobsuche_Details WHERE refnr = ?''', (refnr,))
        count = cursor.fetchone()[0]

        if count == 0: 
        # Insert data into the table    
            cursor.execute('''INSERT INTO Jobsuche_Details (aktuelleVeroeffentlichungsdatum,laufzeitBis,angebotsart,arbeitgeber,
                           branchengruppe,branche,arbeitsorte_land,arbeitsorte_region,arbeitsorte_plz,arbeitsorte_ort,
                           arbeitsorte_koordinaten_lat,arbeitsorte_koordinaten_lon,arbeitszeitmodelle,befristung,uebernahme,
                           betriebsgroesse,eintrittsdatum,ersteVeroeffentlichungsdatum,allianzpartner ,allianzpartnerUrl,titel,beruf,
                           modifikationsTimestamp,stellenbeschreibung,refnr,fuerFluechtlingeGeeignet,nurFuerSchwerbehinderte,
                           anzahlOffeneStellen,arbeitgeber_land,arbeitgeber_region,arbeitgeber_plz, arbeitgeber_ort,arbeitgeber_strasse,
                           arbeitgeber_strasseHausnummer,fertigkeiten_hierarchieName,fertigkeiten_auspraegungen,
                           mobilitaet_reisebereitschaft,fuehrungskompetenzen_hatVollmacht,fuehrungskompetenzen_hatBudgetverantwortung,
                           verguetung,arbeitgeberdarstellungUrl,arbeitgeberdarstellung,hauptDkz,istBetreut,istPrivateArbeitsvermittlung,
                           istZeitarbeit,istGoogleJobsRelevant,anzeigeAnonym
                                                    )
                          VALUES(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)''',
                            (aktuelleVeroeffentlichungsdatum,laufzeitBis,angebotsart,arbeitgeber,branchengruppe,branche,
                            arbeitsorte_land,arbeitsorte_region,arbeitsorte_plz,arbeitsorte_ort,arbeitsorte_koordinaten_lat,
                            arbeitsorte_koordinaten_lon,arbeitszeitmodelle_str,befristung,uebernahme,betriebsgroesse,
                            eintrittsdatum,ersteVeroeffentlichungsdatum,allianzpartner ,allianzpartnerUrl,titel,beruf,modifikationsTimestamp,
                            truncated_stellenbeschreibung,refnr,fuerFluechtlingeGeeignet,nurFuerSchwerbehinderte,anzahlOffeneStellen,
                            arbeitgeber_land,arbeitgeber_region,arbeitgeber_plz, arbeitgeber_ort,arbeitgeber_strasse,arbeitgeber_strasseHausnummer,
                            fertigkeiten_hierarchieName,fertigkeiten_auspraegungen ,mobilitaet_reisebereitschaft,fuehrungskompetenzen_hatVollmacht,
                            fuehrungskompetenzen_hatBudgetverantwortung,verguetung,arbeitgeberdarstellungUrl,truncated_arbeitgeberdarstellung,
                            hauptDkz,istBetreut,istPrivateArbeitsvermittlung,istZeitarbeit,istGoogleJobsRelevant,anzeigeAnonym))
    
    conn.commit()  # Commit once after all insertions are done
    conn.close()



def main_2():
    jwt = get_jwt()

    try:
        all_refnrs = []
        create_table()
        page = 1

        while True:
            result = search2(jwt["access_token"], page)#, "") 
            refnrs_on_page = [item["refnr"] for item in result.get('stellenangebote', [])]
            all_refnrs.extend(refnrs_on_page)

            if not refnrs_on_page:  # If no more results, exit loop
                break

            job_details_data_list = []  # Create a list to store job details

            for refnr in all_refnrs:
                #print(refnr)
                # Fetch job details for each refnr using job_details function
                job_details_data = job_details(jwt["access_token"], refnr)
                job_details_data_list.append(job_details_data)  # Append to the list
                #print(job_details_data)
            
                insert_data_into_db(job_details_data_list)
                
            page += 1
            print("insert data from page for angebotsart:2 ",page)

    except Exception as e:
            
            print(f"Error: {e}")
            # Rollback changes in case of an error
        
    finally:
        # Close the connection
        
        print("Data fetched and saved to SQLite database successfully.")

def main_4():
    jwt = get_jwt()

    try:
        all_refnrs = []
        create_table()
        page = 1

        while True:
            result = search4(jwt["access_token"], page)#, "") 
            refnrs_on_page = [item["refnr"] for item in result.get('stellenangebote', [])]
            all_refnrs.extend(refnrs_on_page)

            if not refnrs_on_page:  # If no more results, exit loop
                break

            job_details_data_list = []  # Create a list to store job details

            for refnr in all_refnrs:
                #print(refnr)
                # Fetch job details for each refnr using job_details function
                job_details_data = job_details(jwt["access_token"], refnr)
                job_details_data_list.append(job_details_data)  # Append to the list
                #print(job_details_data)
            
                insert_data_into_db(job_details_data_list)
                
            page += 1
            print("insert data from page for angebotsart:4 ",page)

    except Exception as e:
            
            print(f"Error: {e}")
            # Rollback changes in case of an error
        
    finally:
        # Close the connection
        
        print("Data fetched and saved to SQLite database successfully.")

def main_34():
    jwt = get_jwt()

    try:
        all_refnrs = []
        create_table()
        page = 1

        while True:
            result = search34(jwt["access_token"], page)#, "") 
            refnrs_on_page = [item["refnr"] for item in result.get('stellenangebote', [])]
            all_refnrs.extend(refnrs_on_page)

            if not refnrs_on_page:  # If no more results, exit loop
                break

            job_details_data_list = []  # Create a list to store job details

            for refnr in all_refnrs:
                #print(refnr)
                # Fetch job details for each refnr using job_details function
                job_details_data = job_details(jwt["access_token"], refnr)
                job_details_data_list.append(job_details_data)  # Append to the list
                #print(job_details_data)
            
                insert_data_into_db(job_details_data_list)
                
            page += 1
            print("insert data from page for angebotsart:34 ",page)

    except Exception as e:
            
            print(f"Error: {e}")
            # Rollback changes in case of an error
        
    finally:
        # Close the connection
        
        print("Data fetched and saved to SQLite database successfully.")

def main_1():
    jwt = get_jwt()

    try:
        all_refnrs = []
        create_table()
        page = 1

        while True:
            result = search1(jwt["access_token"], page)#, "") 
            refnrs_on_page = [item["refnr"] for item in result.get('stellenangebote', [])]
            all_refnrs.extend(refnrs_on_page)

            if not refnrs_on_page:  # If no more results, exit loop
                break

            job_details_data_list = []  # Create a list to store job details

            for refnr in all_refnrs:
                #print(refnr)
                # Fetch job details for each refnr using job_details function
                job_details_data = job_details(jwt["access_token"], refnr)
                job_details_data_list.append(job_details_data)  # Append to the list
                #print(job_details_data)
            
                insert_data_into_db(job_details_data_list)
                
            page += 1
            print("insert data from page for angebotsart:1 ",page)

    except Exception as e:
            
            print(f"Error: {e}")
            # Rollback changes in case of an error
        
    finally:
        # Close the connection
        
        print("Data fetched and saved to SQLite database successfully.")

if __name__ == "__main__":
    with concurrent.futures.ThreadPoolExecutor() as executor:
        future_2 = executor.submit(main_2)
        future_4 = executor.submit(main_4)
        future_34 = executor.submit(main_34)
        future_1 = executor.submit(main_1)
        
        # Wait for all tasks to complete
        concurrent.futures.wait([future_2, future_4, future_34, future_1])



insert data from page for angebotsart:34  2
insert data from page for angebotsart:2  2
insert data from page for angebotsart:4  2
insert data from page for angebotsart:1  2
insert data from page for angebotsart:34  3
insert data from page for angebotsart:2  3
insert data from page for angebotsart:4  3
insert data from page for angebotsart:1  3
insert data from page for angebotsart:34  4
insert data from page for angebotsart:2  4
insert data from page for angebotsart:4  4
insert data from page for angebotsart:1  4
insert data from page for angebotsart:34  5
insert data from page for angebotsart:2  5
insert data from page for angebotsart:4  5
insert data from page for angebotsart:1  5
insert data from page for angebotsart:34  6
insert data from page for angebotsart:2  6
insert data from page for angebotsart:4  6
insert data from page for angebotsart:1  6
insert data from page for angebotsart:34  7
insert data from page for angebotsart:2  7
insert data from page for angebotsart:4  7
inser